In [ ]:
# !export HF_ENDPOINT=https://hf-mirror.com;huggingface-cli download --resume-download OpenGVLab/InternVL2-1B --local-dir InternVL2-1B

In [ ]:
import os

# 提交时可能不能联网，设置成离线模式防止联网失败报错
os.environ['TRANSFORMERS_OFFLINE'] = '1'
os.environ['HF_DATASETS_OFFLINE'] = '1'
os.environ['HF_HUB_OFFLINE'] = '1'

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
path = "/bohr/inter-8ht1/v1"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    low_cpu_mem_usage=True,
    trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

NameError: name 'LlavaForConditionalGeneration' is not defined

In [ ]:
import torch
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=336, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [ ]:
import os
import json
from PIL import Image

if os.environ.get('DATA_PATH_B'):   # 提交时会选择隐藏的测试数据集路径（A+B榜），数据集的格式与A榜数据相同，但数目不同（5360张）
    base_dir = os.environ.get('DATA_PATH_B')
else:
    base_dir = '/bohr/form-recognition-train-b6y2/v4'   # 示例，把A榜测试数据集路径作为测试集路径，仅开发时挂载A榜数据用于debug   # 示例，把A榜测试数据集路径作为测试集路径，仅开发时挂载A榜数据用于debug

In [ ]:
with open(os.path.join(base_dir, 'dataset.json'), 'r') as f:
    data = json.load(f)

with open(os.path.join(base_dir, 'sample_submission.json'), 'r') as f:
    sub = json.load(f)

In [ ]:
generation_config = dict(max_new_tokens=64, do_sample=False)
submission = []
for item in data:
    image_path = os.path.join(base_dir, 'test_images', item["image_path"])
    pixel_values = load_image(image_path, max_num=1).to(torch.bfloat16).cuda()
    question = f'<image>\nThis table caption: "{item["caption"]}". According to the provided table associated with the textual description, which subjuct does the content of the table most likely belong to? Choice one from (Physics,Mathematics,ComputerScience,QuantitativeBiology,QuantitativeFinance,Statistics,ElectricalEngineeringandSystemsScience,Economics)'
    response1, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
    question = '<image>\nHow many rows of this table? give me an int value:'
    response21 = model.chat(tokenizer, pixel_values, question, generation_config)
    question = '<image>\nHow many cols of this table? give me an int value:'
    response22 = model.chat(tokenizer, pixel_values, question, generation_config)
    question = f'<image>\nQuestion: "{item["question"]}"\noptions[0]: "{item["options"][0]}"\noptions[1]: "{item["options"][1]}"\noptions[2]: "{item["options"][2]}"\noptions[3]: "{item["options"][3]}"\nplease choice the correct answer from options (an int value from 0, 1, 2, 3)'
    response3 = model.chat(tokenizer, pixel_values, question, generation_config)
    try:
        if response1 not in ('Physics','Mathematics','ComputerScience','QuantitativeBiology','QuantitativeFinance','Statistics','ElectricalEngineeringandSystemsScience','Economics'):
            for cat in ('Physics','Mathematics','ComputerScience','QuantitativeBiology','QuantitativeFinance','Statistics','ElectricalEngineeringandSystemsScience','Economics'):
                if cat in response1:
                    response1 = cat
                    break
            response1 = 'ComputerScience'
        else:
            pass
    except:
        response1 = 'ComputerScience'
    try:
        rows = int(response21)
    except:
        rows = 6
    try:
        cols = int(response22)
    except:
        cols = 4
    try:
        answer = int(response3[0])
    except:
        answer = 0

    sub_item = {
        "image_path": item["image_path"],
        "category": str(response1),
        "cols": cols,
        "rows": rows,
        "answer": answer,
    }
    submission.append(sub_item)
    print(sub_item)

with open('submission.json', 'w') as f:
    json.dump(submission, f)